# 既約多項式をチェックする
#### 先ずは環境のチェックと **Nemo** などの**Pkg**利用設定

In [1]:
println(versioninfo())
using Nemo,Plots,LaTeXStrings,Latexify,Combinatorics,Dates

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M3
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
nothing

Welcome to Nemo version building

Nemo comes with absolutely no warranty whatsoever


In [2]:
p=19
q=p^2+p+1
Fp=GF(p)
println(p,",",q,", size = ",length(Fp))
println(join(Fp,", "))

19,381, size = 19
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18


In [3]:
Fpx, x = Fp["x"]
IPlst=[]
for a ∈ Fp, b ∈ Fp, c ∈ Fp
    f(x)=x^3+a*x^2+b*x+c
    chk=true
    for e ∈ Fp
        if f(e)==0
            chk=false
            break
        end
    end
    if chk
        push!(IPlst,f(x))
    end
end
if length(IPlst)>20
    println(join(IPlst[1:10],", "),"...",join(IPlst[end-9:end],", "))
else
    println(join(IPlst,", "))
end
println("既約多項式の候補の数 = ",length(IPlst))

x^3 + 2, x^3 + 3, x^3 + 4, x^3 + 5, x^3 + 6, x^3 + 9, x^3 + 10, x^3 + 13, x^3 + 14, x^3 + 15...x^3 + 18*x^2 + 17*x + 6, x^3 + 18*x^2 + 17*x + 10, x^3 + 18*x^2 + 17*x + 14, x^3 + 18*x^2 + 17*x + 18, x^3 + 18*x^2 + 18*x + 2, x^3 + 18*x^2 + 18*x + 6, x^3 + 18*x^2 + 18*x + 8, x^3 + 18*x^2 + 18*x + 9, x^3 + 18*x^2 + 18*x + 11, x^3 + 18*x^2 + 18*x + 15
既約多項式の候補の数 = 2280


In [4]:
Fpt=[]
Fptl=[]
α=0
for k=1:length(IPlst)
    Fpt,α=finite_field(IPlst[k],"α")
    Fptl=[]
    push!(Fptl,1)
    for i=1:length(Fpt)
        e=α^i
#    println(i,": ",e)
        if e==1
            break
        else
            push!(Fptl,e)
        end
    end
    println(k," : size = ",length(Fptl), ", p^3-1 = ",p^3-1)
    if length(Fptl)==p^3-1
    println(Fpt)
        println(k," : ",IPlst[k])
        break
    end
end
println(join(Fptl[1:10],", "),"...",join(Fptl[end-9:end],", "))
pdsl=[0]
for i=1:length(Fpt)
    if coeff(α^i,2)==0
#        println(i,": ",α^i)
        push!(pdsl,i)
    end
end
println(pdsl[1:10],"...",pdsl[end-9:end])
bbl=[]
for i=1:p+1
    push!(bbl,pdsl[i+1]-pdsl[i])
end
println("(",join(bbl,","),")")

1 : size = 27, p^3-1 = 6858
2 : size = 27, p^3-1 = 6858
3 : size = 54, p^3-1 = 6858
4 : size = 54, p^3-1 = 6858
5 : size = 54, p^3-1 = 6858
6 : size = 54, p^3-1 = 6858
7 : size = 27, p^3-1 = 6858
8 : size = 27, p^3-1 = 6858
9 : size = 27, p^3-1 = 6858
10 : size = 27, p^3-1 = 6858
11 : size = 54, p^3-1 = 6858
12 : size = 54, p^3-1 = 6858
13 : size = 762, p^3-1 = 6858
14 : size = 6858, p^3-1 = 6858
Finite field of degree 3 and characteristic 19
14 : x^3 + x + 4
1, α, α^2, 18*α + 15, 18*α^2 + 15*α, 15*α^2 + α + 4, α^2 + 8*α + 16, 8*α^2 + 15*α + 15, 15*α^2 + 7*α + 6, 7*α^2 + 10*α + 16...6*α^2 + α + 1, α^2 + 14*α + 14, 14*α^2 + 13*α + 15, 13*α^2 + α + 1, α^2 + 7*α + 5, 7*α^2 + 4*α + 15, 4*α^2 + 8*α + 10, 8*α^2 + 6*α + 3, 6*α^2 + 14*α + 6, 14*α^2 + 14
[0, 1, 3, 12, 17, 65, 75, 94, 117, 124]...[6640, 6644, 6677, 6748, 6774, 6780, 6814, 6834, 6858, 6859]
(1,2,9,5,48,10,19,23,7,8,13,18,4,33,71,26,6,34,20,24)


In [5]:
cds=[0]
for i=1:length(bbl)
    push!(cds,cds[end]+bbl[i])
end
println("Perfect Difference Set = (",join(cds,","),")")
σ=p^2+p+1
println("σ = ",σ)
otherlst=[]
for i=1:σ-1
#    println(sort!((((cds.*i).-1).%σ).+1))
    cdsa=sort!((((cds.*i).-1).%σ).+1)
    push!(otherlst,cdsa)
end
oblst=[]
for lst ∈ otherlst
    obll=[]
    for i=2:length(lst)
        push!(obll,lst[i]-lst[i-1])
    end
#    sort!(obll)
    while obll[1]>1
        circshift!(obll,-1)
    end
    if obll[2]>obll[end]
        obll=[obll[1];reverse(obll[2:end])]
    end
#    println("(",join(lst,","),") --> (",join(obll,","),")")
    if !(obll ∈ oblst) && !(0 ∈ obll)
        push!(oblst,obll)
    end
end
for bll ∈ oblst
    println("(",join(bll,","),")")
end

Perfect Difference Set = (0,1,3,12,17,65,75,94,117,124,132,145,163,167,200,271,297,303,337,357,381)
σ = 381
(1,2,9,5,48,10,19,23,7,8,13,18,4,33,71,26,6,34,20,24)
(1,7,33,3,26,16,14,9,12,25,27,11,20,96,10,5,13,4,2,47)
(1,18,28,32,52,6,55,11,14,2,13,22,54,5,4,8,26,7,3,20)
(1,22,18,46,17,2,28,3,6,5,10,38,7,13,12,4,72,43,8,26)
(1,10,22,13,25,18,12,39,3,37,4,20,96,6,2,7,14,5,31,16)
(1,2,40,6,14,47,5,7,9,8,10,77,31,33,11,26,4,15,13,22)
(1,7,24,5,21,14,9,2,52,13,3,17,10,12,6,41,15,4,34,91)
(1,4,14,10,7,6,3,48,2,32,43,12,15,46,8,30,22,11,47,20)
(1,7,45,48,47,17,3,13,26,12,18,10,4,5,6,35,27,34,2,21)
(1,6,3,28,14,4,12,17,24,36,50,26,39,5,20,2,13,8,11,62)
(1,9,14,18,11,5,3,17,13,26,44,2,4,65,15,12,28,7,66,21)
(1,5,19,7,40,28,8,12,10,23,16,18,3,14,27,2,9,4,50,85)
(1,23,17,2,8,3,25,20,14,12,6,15,16,60,5,4,35,22,7,86)
(1,4,3,38,42,20,6,48,16,9,47,19,11,33,32,2,15,12,10,13)
(1,3,5,12,11,26,61,24,10,56,6,7,33,2,36,15,14,16,25,18)
(1,3,23,19,2,14,17,24,36,13,7,8,10,12,59,11,54,29,34,5)
(1,16,22,33,91,2

In [6]:
n=2
println("p = ",p,", n = ",n)
Fp,α=finite_field(p,n,"α")
for e ∈ Fp
    print(e,", ")
end

p = 19, n = 2
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, α, α + 1, α + 2, α + 3, α + 4, α + 5, α + 6, α + 7, α + 8, α + 9, α + 10, α + 11, α + 12, α + 13, α + 14, α + 15, α + 16, α + 17, α + 18, 2*α, 2*α + 1, 2*α + 2, 2*α + 3, 2*α + 4, 2*α + 5, 2*α + 6, 2*α + 7, 2*α + 8, 2*α + 9, 2*α + 10, 2*α + 11, 2*α + 12, 2*α + 13, 2*α + 14, 2*α + 15, 2*α + 16, 2*α + 17, 2*α + 18, 3*α, 3*α + 1, 3*α + 2, 3*α + 3, 3*α + 4, 3*α + 5, 3*α + 6, 3*α + 7, 3*α + 8, 3*α + 9, 3*α + 10, 3*α + 11, 3*α + 12, 3*α + 13, 3*α + 14, 3*α + 15, 3*α + 16, 3*α + 17, 3*α + 18, 4*α, 4*α + 1, 4*α + 2, 4*α + 3, 4*α + 4, 4*α + 5, 4*α + 6, 4*α + 7, 4*α + 8, 4*α + 9, 4*α + 10, 4*α + 11, 4*α + 12, 4*α + 13, 4*α + 14, 4*α + 15, 4*α + 16, 4*α + 17, 4*α + 18, 5*α, 5*α + 1, 5*α + 2, 5*α + 3, 5*α + 4, 5*α + 5, 5*α + 6, 5*α + 7, 5*α + 8, 5*α + 9, 5*α + 10, 5*α + 11, 5*α + 12, 5*α + 13, 5*α + 14, 5*α + 15, 5*α + 16, 5*α + 17, 5*α + 18, 6*α, 6*α + 1, 6*α + 2, 6*α + 3, 6*α + 4, 6*α + 5, 6*α + 6, 6*α + 7, 6*α + 8, 6

In [7]:
start = Dates.now()
Fpx, x = Fp["x"]
IPlst=[]
for a ∈ Fp, b ∈ Fp, c ∈ Fp
    f(x)=x^3+a*x^2+b*x+c
    chk=true
    for e ∈ Fp
        if f(e)==0
            chk=false
            break
        end
    end
    if chk
        push!(IPlst,f(x))
    end
    if length(IPlst)>20
        break
    end
end
println(length(IPlst))
println(join(IPlst[1:10],",")," ... ",join(IPlst[end-9:end],","))
dtime = Dates.now() - start
println("かかった時間: $dtime")

21
x^3 + 2,x^3 + 3,x^3 + 4,x^3 + 5,x^3 + 6,x^3 + 9,x^3 + 10,x^3 + 13,x^3 + 14,x^3 + 15 ... x^3 + 17,x^3 + α,x^3 + α + 1,x^3 + α + 3,x^3 + α + 4,x^3 + α + 5,x^3 + α + 6,x^3 + α + 8,x^3 + α + 9,x^3 + α + 10
かかった時間: 15 milliseconds


In [8]:
start = Dates.now()
ω=0
for k=1:length(IPlst)
    Fpt,ω=finite_field(IPlst[k],"ω")
    Fptl=[]
    push!(Fptl,1)
    for i=1:length(Fpt)
#    e=ω^i
        e=(ω+3)^i
#    println(i,": ",e)
        if e==1
            break
        else
            push!(Fptl,e)
        end
    end
    println(k," : size = ",length(Fptl), ", p^3-1 = ",(p^n)^3-1)
    if length(Fptl)==(p^n)^3-1
        println(Fpt)
        println(k," : ",IPlst[k])
        break
    end
end
println(join(Fptl[1:10],", "))
pdsl=[0]
for i=1:length(Fpt)
    if coeff((ω+3)^i,2)==0
#    if coeff(ω^i,2)==0
#        println(i,": ",α^i)
        push!(pdsl,i)
    end
end
println(join(pdsl[1:10],","),"...",join(pdsl[end-9:end],","))
bbl=[]
for i=1:p^n+1
    push!(bbl,pdsl[i+1]-pdsl[i])
end
println(join(bbl,","))
println("size = ",length(bbl))
dtime = Dates.now() - start
println("かかった時間: $dtime")

1 : size = 3429, p^3-1 = 47045880
2 : size = 3429, p^3-1 = 47045880
3 : size = 3429, p^3-1 = 47045880
4 : size = 6858, p^3-1 = 47045880
5 : size = 6858, p^3-1 = 47045880
6 : size = 762, p^3-1 = 47045880
7 : size = 3429, p^3-1 = 47045880
8 : size = 6858, p^3-1 = 47045880
9 : size = 6858, p^3-1 = 47045880
10 : size = 2286, p^3-1 = 47045880
11 : size = 1143, p^3-1 = 47045880
12 : size = 6858, p^3-1 = 47045880
13 : size = 1306830, p^3-1 = 47045880
14 : size = 5880735, p^3-1 = 47045880
15 : size = 47045880, p^3-1 = 47045880
Finite field of degree 3 over Fp
15 : x^3 + α + 3
1, ω + 3, ω^2 + 6*ω + 9, 9*ω^2 + 8*ω + 18*α + 5, 16*ω^2 + (18*α + 10)*ω + 7*α + 7, (18*α + 1)*ω^2 + (4*α + 18)*ω + 5*α + 11, (α + 2)*ω^2 + (17*α + 8)*ω + 18*α + 9, (α + 14)*ω^2 + (12*α + 14)*ω + 10*α + 4, (15*α + 18)*ω^2 + (8*α + 8)*ω + 12*α + 10, (15*α + 5)*ω^2 + (17*α + 15)*ω + 15*α + 6
0,1,19,361,980,1063,1888,2433,3087,3392...47044165,47044373,47045131,47045321,47045470,47045518,47045648,47045685,47045880,47045881
1,1

In [9]:
start = Dates.now()
bblw=[bbl;bbl]
#println(bblw)
suml=[]
for b=1:length(bbl)-1
    for i=1:length(bbl)
        sum=0
        for j=i:i+b-1
            sum=sum+bblw[j]
        end
        push!(suml,sum)
    end
end
push!(suml,sum(bbl))
println(join(sort(unique(suml))[1:20],",")," ... ",join(sort(unique(suml))[end-10:end],",")," : ",length(suml))
dtime = Dates.now() - start
println("かかった時間: $dtime")

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20 ... 130673,130674,130675,130676,130677,130678,130679,130680,130681,130682,130683 : 130683
かかった時間: 1287 milliseconds


In [10]:
cds=[0]
for i=1:length(bbl)
    push!(cds,cds[end]+bbl[i])
end
println("Perfect Difference Set = (",join(cds,","),")")
σ=(p^n)^2+(p^n)+1
println("σ = ",σ)
otherlst=[]
for i=1:σ-1
#    println(sort!((((cds.*i).-1).%σ).+1))
    cdsa=sort!((((cds.*i).-1).%σ).+1)
    push!(otherlst,cdsa)
end
oblst=[]
for lst ∈ otherlst
    obll=[]
    for i=2:length(lst)
        push!(obll,lst[i]-lst[i-1])
    end
#    sort!(obll)
    while obll[1]>1
        circshift!(obll,-1)
    end
    if obll[2]>obll[end]
        obll=[obll[1];reverse(obll[2:end])]
    end
#    println("(",join(lst,","),") --> (",join(obll,","),")")
    if !(obll ∈ oblst) && !(0 ∈ obll)
        push!(oblst,obll)
    end
end
println(length(oblst))
if length(oblst)>20
    for bll ∈ oblst[1:10]
        println("(",join(bll,","),")")
    end
    println("...")
    for bll ∈ oblst[end-9:end]
        println("(",join(bll,","),")")
    end
end


Perfect Difference Set = (0,1,19,361,980,1063,1888,2433,3087,3392,3586,3911,4051,4154,4319,4608,4718,4741,4952,5191,5238,5847,6859,6934,7243,7361,8698,8960,9092,9141,9147,9176,9495,9772,10065,10236,10343,10951,11433,11436,11639,12015,12175,12622,12952,14358,14754,15137,15268,15594,15763,15964,16456,16838,17055,17737,18620,18960,19823,19839,19890,20197,20263,20840,21681,21731,22802,23062,23555,23656,24240,24876,24898,26237,27500,27970,28153,28726,28788,29512,29724,29937,30783,31953,32345,32550,32662,32673,32821,32826,33262,33384,33483,34035,34300,34380,34579,34698,34890,34920,34929,34992,35211,35534,35630,35872,36072,36938,37289,37584,37996,38131,38712,39395,39557,40502,40817,41189,41544,41950,42024,42065,42996,43110,43400,43561,43576,43661,43846,44389,45421,45461,45641,45758,46071,46129,46227,46745,46931,47660,48365,48407,48976,49044,49722,49866,49897,51298,51324,52580,53508,53886,54194,54346,54985,54995,55076,55496,55637,55828,56987,57415,58556,58653,59293,59567,59881,60288,60375,6055